In [9]:
import copernicusmarine
import xarray as xr
import fsspec
import pandas as pd
from seapopym.standard.units import StandardUnitsLabels

## Load DATA


### Observations


In [10]:
papa_obs = xr.open_dataset("../data/1_products/Papa_obs.zarr", engine="zarr")
papa_obs

<xarray.Dataset> Size: 3MB
Dimensions:    (time: 623, latitude: 12, longitude: 26, layer: 1)
Coordinates:
  * latitude   (latitude) float64 96B 46.5 47.5 48.5 49.5 ... 55.5 56.5 57.5
  * layer      (layer) int64 8B 0
  * longitude  (longitude) float64 208B -156.5 -153.5 -152.5 ... -129.5 -128.5
  * time       (time) datetime64[ns] 5kB 1995-09-24 1995-09-25 ... 2020-08-31
Data variables:
    day        (time, latitude, longitude, layer) float64 2MB ...
    night      (time, latitude, longitude, layer) float64 2MB ...

In [12]:
papa_forcing = xr.open_dataset(
    "/Users/adm-lehodey/Documents/Workspace/Data/phd/FORCING/Papa_forcing.zarr", engine="zarr"
)
papa_forcing

<xarray.Dataset> Size: 197MB
Dimensions:              (time: 8280, depth: 3, latitude: 12, longitude: 31)
Coordinates:
  * depth                (depth) int32 12B 1 2 3
  * latitude             (latitude) float64 96B 46.5 47.5 48.5 ... 56.5 57.5
  * longitude            (longitude) float64 248B -157.5 -156.5 ... -127.5
  * time                 (time) datetime64[ns] 66kB 1998-01-01 ... 2020-09-01
Data variables:
    T                    (time, depth, latitude, longitude) float64 74MB ...
    npp                  (time, latitude, longitude) float64 25MB ...
    pelagic_layer_depth  (time, depth, latitude, longitude) float64 74MB ...
    zooc                 (time, latitude, longitude) float64 25MB ...
Attributes:
    Conventions:  CF-1.7
    history:      Created on 2024-09-6
    institution:  CLS
    references:   http://www.cls.fr; http://www.seapodym.eu
    source:       SEAPODYM-LMTL 3.0.0
    title:        Physical forcings of global ocean low and mid trophic level...

In [13]:
common_latitudes = sorted(set(papa_obs.latitude.data).intersection(set(papa_forcing.latitude.data)))
common_longitudes = sorted(set(papa_obs.longitude.data).intersection(set(papa_forcing.longitude.data)))

print(f"Latitudes communes: {common_latitudes}")
print(f"Longitudes communes: {common_longitudes}")

# Sélection des données de papa_forcing avec les coordonnées communes
papa_forcing_common = papa_forcing.sel(latitude=common_latitudes, longitude=common_longitudes)
papa_forcing_common

Latitudes communes: [46.5, 47.5, 48.5, 49.5, 50.5, 51.5, 52.5, 53.5, 54.5, 55.5, 56.5, 57.5]
Longitudes communes: [-156.5, -153.5, -152.5, -151.5, -150.5, -149.5, -147.5, -146.5, -145.5, -144.5, -143.5, -142.5, -141.5, -140.5, -139.5, -138.5, -137.5, -136.5, -135.5, -134.5, -133.5, -132.5, -131.5, -130.5, -129.5, -128.5]


<xarray.Dataset> Size: 165MB
Dimensions:              (time: 8280, depth: 3, latitude: 12, longitude: 26)
Coordinates:
  * depth                (depth) int32 12B 1 2 3
  * latitude             (latitude) float64 96B 46.5 47.5 48.5 ... 56.5 57.5
  * longitude            (longitude) float64 208B -156.5 -153.5 ... -128.5
  * time                 (time) datetime64[ns] 66kB 1998-01-01 ... 2020-09-01
Data variables:
    T                    (time, depth, latitude, longitude) float64 62MB ...
    npp                  (time, latitude, longitude) float64 21MB ...
    pelagic_layer_depth  (time, depth, latitude, longitude) float64 62MB ...
    zooc                 (time, latitude, longitude) float64 21MB ...
Attributes:
    Conventions:  CF-1.7
    history:      Created on 2024-09-6
    institution:  CLS
    references:   http://www.cls.fr; http://www.seapodym.eu
    source:       SEAPODYM-LMTL 3.0.0
    title:        Physical forcings of global ocean low and mid trophic level...

## Change the depth to be categorical AND starting with 0 value rather than 1


In [15]:
depth_attrs = {
    "flag_values": "[0 1 2]",
    "flag_meanings": "epipelagic, upper-mesopelagic, lower-mesopelagic",
    "desc": "The layer depth according to Seapodym definition",
    "standard_name": "depth",
    "long_name": "Layer depth",
    "axis": "Z",
}
papa_forcing_common = papa_forcing_common.assign_coords(depth=[0, 1, 2])
papa_forcing_common.depth.attrs = depth_attrs
papa_forcing_common

<xarray.Dataset> Size: 165MB
Dimensions:              (time: 8280, depth: 3, latitude: 12, longitude: 26)
Coordinates:
  * latitude             (latitude) float64 96B 46.5 47.5 48.5 ... 56.5 57.5
  * longitude            (longitude) float64 208B -156.5 -153.5 ... -128.5
  * time                 (time) datetime64[ns] 66kB 1998-01-01 ... 2020-09-01
  * depth                (depth) int64 24B 0 1 2
Data variables:
    T                    (time, depth, latitude, longitude) float64 62MB ...
    npp                  (time, latitude, longitude) float64 21MB ...
    pelagic_layer_depth  (time, depth, latitude, longitude) float64 62MB ...
    zooc                 (time, latitude, longitude) float64 21MB ...
Attributes:
    Conventions:  CF-1.7
    history:      Created on 2024-09-6
    institution:  CLS
    references:   http://www.cls.fr; http://www.seapodym.eu
    source:       SEAPODYM-LMTL 3.0.0
    title:        Physical forcings of global ocean low and mid trophic level...

## Gathering Data & Save


In [16]:
papa_forcing_common.to_zarr("../data/1_products/Papa_cmems.zarr", mode="w")